In [1]:
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn.utils import parameters_to_vector, vector_to_parameters

from pymoo.core.problem import ElementwiseProblem

import column_names
from engine import Signal, Spot, Trader
import usdt

In [28]:
class NNTrader(Trader):
	def __init__(self, model: nn.Module, max_pos_size: usdt = 100, warmup: int = 600):
		super().__init__()

		self.model = model
		self.max_pos_size = max_pos_size
		self.warmup = warmup

	@torch.inference_mode()
	def __call__(self, *, data_so_far: pd.DataFrame, engine: Spot) -> Signal:
		# feed the model the ratio of the last two bars Open
		ratio: float = data_so_far.iloc[-1]['Open'] / data_so_far.iloc[-2]['Open']
		ratio = math.log(ratio)
		ratio = torch.tensor([[ratio]], dtype=torch.float64)
		signal = self.model(ratio)

		# ignore the first warmup bars
		if len(data_so_far) < self.warmup: return Signal.HOLD

		# do not trade if the position is too big
		if engine.position and engine.position.tot_price >= self.max_pos_size:
			return Signal.HOLD

		if signal > +0.5: return Signal.BUY
		if signal < -0.5: return Signal.SELL

		return Signal.HOLD


class Problem(ElementwiseProblem):
	@torch.inference_mode()
	def __init__(self, model: nn.Module, data: pd.DataFrame, *args, **kwargs):
		super().__init__(
			n_var = len(parameters_to_vector(model.parameters())),
			n_obj = 2,
			n_constr = 0,
			xl = -10,
			xu = +10,
			*args,
			**kwargs,
		)

		self.model = model
		self.data = data

	@torch.inference_mode()
	def _evaluate(self, x: np.ndarray, out, *args, **kwargs):
		self.model.h = None # reset hidden state
		vector_to_parameters(torch.from_numpy(x), self.model.parameters())

		engine = Spot()
		trader = NNTrader(self.model, max_pos_size=100)
		engine.trade(trader=trader, data=self.data, min_order_size=5, initial_quote=200)

		# since we are doing minimization, multiply quote by -1
		out['F'] = [-1 * engine.quote, engine.mdd]


class Model(nn.Module):
	def __init__(self, input_size=1, hidden_size=10, num_layers=5, bias=True):
		super().__init__()
		self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, bias=bias)
		self.linear = nn.Linear(hidden_size, 1, bias=bias)
		self.h = None # hidden state is preserved between calls

	def forward(self, x):
		x, self.h = self.lstm(x, self.h)
		x = self.linear(x)
		return nn.functional.tanh(x)

with torch.inference_mode():
	model = Model(input_size=1, hidden_size=5, num_layers=2, bias=False)
	model.compile()

len(parameters_to_vector(model.parameters()))

325

In [91]:
!mkdir -p data
!wget -nc https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1s/XRPUSDT-1s-2025-02.zip -O data/XRPUSDT-1s-2025-02.zip

File ‘data/XRPUSDT-1s-2025-02.zip’ already there; not retrieving.


In [30]:
df = pd.read_csv(
	'data/XRPUSDT-1s-2025-02.zip',
	names = column_names.names,
	index_col = 'Open time',
	usecols = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume'],
	skiprows=23 * 60 * 60,
	nrows=2 * 60 * 60,
)
df.index = pd.to_datetime(df.index, unit='us')

In [31]:
import multiprocessing

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import StarmapParallelization
from pymoo.core.termination import NoTermination
from pymoo.operators.crossover.ux import UniformCrossover

pool = multiprocessing.Pool()
runner = StarmapParallelization(pool.starmap)

problem = Problem(model=model, data=df, elementwise_runner=runner)

algorithm = NSGA2(
	pop_size=20,
	crossover=UniformCrossover(prob=1.0),
)
algorithm.setup(problem, termination=NoTermination())

In [88]:
# rerun for as many generations as you want
pop = algorithm.ask()
algorithm.evaluator.eval(problem, pop)
algorithm.tell(infills=pop)
res = algorithm.result()

In [89]:
print(res.F)

[[-2.00002786e+02  4.99806038e-05]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00001753e+02  1.92846618e-05]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]
 [-2.00000000e+02  0.00000000e+00]]


In [90]:
np.savetxt('results.txt', res.X[:1], delimiter=', ')